In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
import requests

First, import the data from CSVs and clean it.

In [ ]:
# read in csvs
df_2016 = pd.read_csv("2016-stats.csv")
df_2017 = pd.read_csv("2017-stats.csv")

# get rid of goalies from both dfs
df_2016 = df_2016[df_2016.POS != "G"]
df_2017 = df_2017[df_2017.POS != "G"]

# get points from 2017 stats
df_2017["PTS"] = df_2017["G"]*10 + df_2017["A"]*6 + df_2017["CC"]*1.5 + df_2017["S"] + df_2017["SOG"] + df_2017["ACR"]*.75 + df_2017["CR"]*.25 + df_2017["PKG"]*-2 + df_2017["P"]*.15 + df_2017["TKL"] + df_2017["INT"]*.5 + df_2017["Y"]*-1.5 + df_2017["YR"]*-1.5 + df_2017["R"]*-3

# add 2017 points as column in 2016 df
data = pd.merge(df_2016, df_2017[["Player Name", "PTS"]], on="Player Name")
data = data.drop(columns=["Team"])

# get rid of any duplicates
data = data.drop_duplicates(subset=["Player Name"])

# scrape for ages and add them
data["Age"] = 0

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

ages = []
for index, row in data.iterrows():
    name = row["Player Name"]
    print(name)
    name = name.replace(" ", "+")
    url = "https://www.google.com/search?q=" + name + "+age"
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    elements = soup.select("div.Z0LcW")
    if len(elements) == 0:
        ages.append(0)
        continue
        
    text = soup.select("div.Z0LcW")[0].text
    age = int(text.split("\xa0")[0])
    ages.append(age)
    
data["Age"] = ages
data = data[data.Age != 0]

# break up stats by position
d_data = data[data.POS.isin(["D", "M/D"])]
m_data = data[data.POS.isin(["M"])]
f_data = data[data.POS.isin(["F", "F/M"])]


                      Player Name  POS  GP   MIN  ST  ON  OFF   Y  YR  R  \
0               Cesar Azpilicueta  M/D  38  3420  38   0    0   4   0  0   
1                      Steve Cook    D  38  3420  38   0    0   5   0  0   
2                       Joel Ward    D  38  3418  38   0    1   7   0  0   
3                Gylfi Sigurdsson    M  38  3328  37   1    2   2   0  0   
4                 Nathaniel Clyne    D  37  3324  37   0    1   0   0  0   
5                   Jermain Defoe    F  37  3322  37   0    2   1   0  0   
6                    Craig Dawson    D  37  3278  37   0    2  10   0  0   
7                 Darren Fletcher    M  38  3235  37   1    5   0   0  0   
8                    Stephen Ward    D  37  3232  37   0    3   5   0  0   
9                  Etienne Capoue    M  37  3212  37   0   10   5   0  0   
10                    Gary Cahill    D  37  3296  36   1    1   5   0  0   
11                  Romelu Lukaku    F  37  3267  36   1    2   3   0  0   
12          

Steve Cook
Joel Ward
Gylfi Sigurdsson
Nathaniel Clyne
Jermain Defoe
Craig Dawson
Darren Fletcher
Stephen Ward
Etienne Capoue
Gary Cahill
Romelu Lukaku
Matthew Lowton
Alexis Sanchez
Samuel Clucas
Christian Eriksen
James Milner
Gareth McAuley
Christian Benteke
Eden Hazard
Ashley Williams
Nacho Monreal
Erik Pieters
Ryan Shawcross
Michael Keane
Bruno Martins Indi
N'Golo Kante
Christian Fuchs
Jason Puncheon
Diego Costa
Oriol Romeu
Dele Alli
Victor Wanyama
Adam Smith
Roberto Firmino
Charlie Daniels
Eric Dier
Wilfried Zaha
Ben Mee
Danny Simpson
Simon Francis
Joe Allen
Georginio Wijnaldum
Harry Arter
David Luiz
Jose Holebas
Jan Vertonghen
Kevin De Bruyne
Riyad Mahrez
Laurent Koscielny
Jamie Vardy
Nathan Redmond
Ross Barkley
Salomon Rondon
Mesut Ozil
Leighton Baines
Marten De Roon
Sebastian Prodl
Marko Arnautovic
Idrissa Gueye
Troy Deeney
David Silva
Fernandinho
Andrew Robertson
Cheikhou Kouyate
Joshua King
Kyle Naughton
Manuel Lanzini
Kyle Walker
Jeff Hendrick
Marcos Alonso
Nemanja Matic
Jonny

Start with a simple regression for each position on all the statistical categories.

In [25]:
lm = linear_model.LinearRegression()

# linear regression for forwards
# create training and testing data
xf = f_data.drop(columns=["PTS"])
yf = f_data.PTS

xf_train, xf_test, yf_train, yf_test = train_test_split(xf, yf, test_size=0.2)

